In [4]:
import numpy as np
import pandas as pd
import geopandas as gpd
from sklearn.neighbors import KNeighborsClassifier

change_type_map = {'Demolition': 0, 'Road': 1, 'Residential': 2, 'Commercial': 3, 'Industrial': 4, 'Mega Projects': 5}

In [5]:
# Read csvs
train_df = gpd.read_file('data/train.geojson', index_col=0)

## Analyzing Data

In [6]:
# Count number of each change type
change_type_counts = train_df['change_type'].value_counts()
print(change_type_counts)

change_type
Residential      148435
Commercial       100422
Demolition        31509
Road              14305
Industrial         1324
Mega Projects       151
Name: count, dtype: int64


In [7]:
# Count number of missing values in each column
missing_values = train_df.isnull().sum()
print(missing_values)

urban_type                 0
geography_type             0
change_type                0
img_red_mean_date1      1954
img_green_mean_date1    1954
img_blue_mean_date1     1954
img_red_std_date1       1954
img_green_std_date1     1954
img_blue_std_date1      1954
img_red_mean_date2      1954
img_green_mean_date2    1954
img_blue_mean_date2     1954
img_red_std_date2       1954
img_green_std_date2     1954
img_blue_std_date2      1954
img_red_mean_date3      1954
img_green_mean_date3    1954
img_blue_mean_date3     1954
img_red_std_date3       1954
img_green_std_date3     1954
img_blue_std_date3      1954
img_red_mean_date4      1954
img_green_mean_date4    1954
img_blue_mean_date4     1954
img_red_std_date4       1954
img_green_std_date4     1954
img_blue_std_date4      1954
img_red_mean_date5      1954
img_green_mean_date5    1954
img_blue_mean_date5     1954
img_red_std_date5       1954
img_green_std_date5     1954
img_blue_std_date5      1954
date0                   1383
change_status_

In [8]:
# Count number of missing values in each column for each change type
none_counts = train_df.groupby('change_type').apply(lambda x: x.isnull().sum())
print(none_counts)

               urban_type  geography_type  change_type  img_red_mean_date1   
change_type                                                                  
Commercial              0               0            0                 509  \
Demolition              0               0            0                 242   
Industrial              0               0            0                   1   
Mega Projects           0               0            0                   0   
Residential             0               0            0                1163   
Road                    0               0            0                  39   

               img_green_mean_date1  img_blue_mean_date1  img_red_std_date1   
change_type                                                                   
Commercial                      509                  509                509  \
Demolition                      242                  242                242   
Industrial                        1                    1   

In [9]:
# Count number of missing values in each column for each change type as a percentage
none_counts = train_df.groupby('change_type').apply(lambda x: x.isnull().sum() / len(x))
print(none_counts)

               urban_type  geography_type  change_type  img_red_mean_date1   
change_type                                                                  
Commercial            0.0             0.0          0.0            0.005069  \
Demolition            0.0             0.0          0.0            0.007680   
Industrial            0.0             0.0          0.0            0.000755   
Mega Projects         0.0             0.0          0.0            0.000000   
Residential           0.0             0.0          0.0            0.007835   
Road                  0.0             0.0          0.0            0.002726   

               img_green_mean_date1  img_blue_mean_date1  img_red_std_date1   
change_type                                                                   
Commercial                 0.005069             0.005069           0.005069  \
Demolition                 0.007680             0.007680           0.007680   
Industrial                 0.000755             0.000755   

The ratio is very light, it is possible to fill the lines with missing data with the mean value.

In [ ]:
# fill missing values for columns with numbers or dates with the mean of the column
for column in train_df.columns:
    if train_df[column].dtype == np.float64 or train_df[column].dtype == np.int64:
        train_df[column] = train_df[column].fillna(train_df[column].mean())
    if train_df[column].dtype == np.datetime64:
        train_df[column] = train_df[column].fillna(train_df[column].mean())
    if train_df[column].dtype == np.object_:
        train_df[column] = train_df[column].fillna(train_df[column].mode()[0])

# count number of missing values in each column
missing_values = train_df.isnull().sum()
print(missing_values)

urban_type              0
geography_type          0
change_type             0
img_red_mean_date1      0
img_green_mean_date1    0
img_blue_mean_date1     0
img_red_std_date1       0
img_green_std_date1     0
img_blue_std_date1      0
img_red_mean_date2      0
img_green_mean_date2    0
img_blue_mean_date2     0
img_red_std_date2       0
img_green_std_date2     0
img_blue_std_date2      0
img_red_mean_date3      0
img_green_mean_date3    0
img_blue_mean_date3     0
img_red_std_date3       0
img_green_std_date3     0
img_blue_std_date3      0
img_red_mean_date4      0
img_green_mean_date4    0
img_blue_mean_date4     0
img_red_std_date4       0
img_green_std_date4     0
img_blue_std_date4      0
img_red_mean_date5      0
img_green_mean_date5    0
img_blue_mean_date5     0
img_red_std_date5       0
img_green_std_date5     0
img_blue_std_date5      0
date0                   0
change_status_date0     0
date1                   0
change_status_date1     0
date2                   0
change_statu

## Dates Filtering

### Ordering dates

In [14]:
# Create the table of train data
train_dates = train_df[[f'date{i}' for i in range(0, 5)]]
print(train_dates)

             date0       date1       date2       date3       date4
0       01-08-2018  09-12-2013  10-09-2016  22-07-2019  24-07-2017
1       01-08-2018  09-12-2013  10-09-2016  22-07-2019  24-07-2017
2       01-08-2018  09-12-2013  10-09-2016  22-07-2019  24-07-2017
3       01-08-2018  09-12-2013  10-09-2016  22-07-2019  24-07-2017
4       01-08-2018  09-12-2013  10-09-2016  22-07-2019  24-07-2017
...            ...         ...         ...         ...         ...
296141  19-11-2014  25-02-2017  27-01-2014  28-03-2018  28-12-2015
296142  19-11-2014  25-02-2017  27-01-2014  28-03-2018  28-12-2015
296143  19-11-2014  25-02-2017  27-01-2014  28-03-2018  28-12-2015
296144  19-11-2014  25-02-2017  27-01-2014  28-03-2018  28-12-2015
296145  19-11-2014  25-02-2017  27-01-2014  28-03-2018  28-12-2015

[296146 rows x 5 columns]


In [15]:
# Calculating arsort indexes of dates
def index_sort_by_dates(date_row):
    return np.argsort(pd.to_datetime(date_row, format='%d-%m-%Y'))

# Test index_sort_by_dates
print(index_sort_by_dates(['18-09-2010', '18-09-2009', '18-09-2000']))

[2 1 0]


In [17]:
# Create the table of arsort indexes of dates
train_index = train_dates[[f'date{i}' for i in range(0, 5)]].apply(index_sort_by_dates, axis=1)
print(train_index)

        date0  date1  date2  date3  date4
0           1      2      4      0      3
1           1      2      4      0      3
2           1      2      4      0      3
3           1      2      4      0      3
4           1      2      4      0      3
...       ...    ...    ...    ...    ...
296141      2      0      4      1      3
296142      2      0      4      1      3
296143      2      0      4      1      3
296144      2      0      4      1      3
296145      2      0      4      1      3

[296146 rows x 5 columns]


### Times Intervals

In [18]:
def time_interval(row):
    date_row = row[:5]
    index_row = row[5:]
    return [int((pd.to_datetime(date_row[index_row[i + 1]], format='%d-%m-%Y') - pd.to_datetime(date_row[index_row[i]], format='%d-%m-%Y')).days) if date_row[index_row[i]] is not None and date_row[index_row[i + 1]] is not None else None for i in range(4)]

# Test time_interval
print(time_interval(['18-09-2020', '18-09-2009', '18-09-2010', '18-09-2011', '18-09-2012', 1, 2, 3, 4, 0]))

[365, 365, 366, 2922]


In [19]:
# Create the table of time intervals
train_intervals = pd.concat([train_dates, train_index], axis=1).apply(time_interval, axis=1)
train_intervals = pd.DataFrame(train_intervals.tolist(), columns=['time_interval1', 'time_interval2', 'time_interval3', 'time_interval4'])
print(train_intervals)

        time_interval1  time_interval2  time_interval3  time_interval4
0                 1006             317             373             355
1                 1006             317             373             355
2                 1006             317             373             355
3                 1006             317             373             355
4                 1006             317             373             355
...                ...             ...             ...             ...
296141             296             404             425             396
296142             296             404             425             396
296143             296             404             425             396
296144             296             404             425             396
296145             296             404             425             396

[296146 rows x 4 columns]


In [20]:
# Count number of None
none_counts = train_intervals.isnull().sum()
print(none_counts)

time_interval1    0
time_interval2    0
time_interval3    0
time_interval4    0
dtype: int64


In [21]:
# Creating array
train_intervals = np.asarray(train_intervals).reshape(-1, 4)
print(train_intervals.shape)
print(train_intervals)

(296146, 4)
[[1006  317  373  355]
 [1006  317  373  355]
 [1006  317  373  355]
 ...
 [ 296  404  425  396]
 [ 296  404  425  396]
 [ 296  404  425  396]]


## Colors Filtering

### Red Mean

In [23]:
# Define the get_ordered_values function
def get_ordered_values(row):
    data = row[:5]
    indexes = row[5:]
    return [data[int(indexes[i])] for i in range(5)]

# Test get_ordered_values
print(get_ordered_values([1, 2, 3, 4, 5, 4, 3, 2, 1, 0]))

[5, 4, 3, 2, 1]


### Red-Green-Blue Means and Svd

In [24]:
config = {'red_mean', 'blue_mean', 'green_mean', 'red_std', 'blue_std', 'green_std'}
train_colors = pd.DataFrame()

for element in config:
    train_element = pd.concat([train_df[[f'img_{element}_date{i}' for i in range(1, 6)]], train_index], axis=1).apply(get_ordered_values, axis=1)
    train_element = pd.DataFrame(train_element.tolist(), columns=[f'img_{element}_date{i}' for i in range(1, 6)])
    train_element = train_element.fillna(train_element.mean())
    train_element = (train_element - train_element.mean()) / train_element.std()
    train_colors = pd.concat([train_colors, train_element], axis=1)

print(train_colors)

        img_red_std_date1  img_red_std_date2  img_red_std_date3   
0                0.014411           1.975850           0.304269  \
1               -0.300026           1.035275           0.095744   
2                0.061137           2.161605           0.345593   
3                0.413590           1.651254           0.136546   
4               -0.248410           0.339109           0.756329   
...                   ...                ...                ...   
296141          -0.405247          -0.081253           0.676931   
296142           0.561150           2.002278           1.856386   
296143          -0.795748          -0.387250          -0.104333   
296144           0.113774           0.359181           1.914090   
296145           0.123668           0.949116           1.186471   

        img_red_std_date4  img_red_std_date5  img_blue_std_date1   
0                0.123766           0.883730           -0.027335  \
1               -0.203617           0.699843           -0.3

In [21]:
# Creating array
train_colors = np.asarray(train_colors).reshape(-1, 30)
print(train_colors.shape)

(296146, 30)


## Hot-one Encoding for Urban and Geographical data

### Change Status

In [22]:
# Get change_status by ascending dates
train_change_status = pd.concat([train_df[[f'change_status_date{i}' for i in range(0, 5)]], train_index], axis=1).apply(get_ordered_values, axis=1)
print(train_change_status)

0         [Greenland, Construction Started, Construction...
1         [Greenland, Land Cleared, Construction Midway,...
2         [Greenland, Land Cleared, Land Cleared, Constr...
3         [Greenland, Construction Started, Construction...
4         [Prior Construction, Prior Construction, Prior...
                                ...                        
296141    [Land Cleared, Greenland, Land Cleared, Constr...
296142    [Greenland, Greenland, Land Cleared, Construct...
296143    [Greenland, Greenland, Greenland, Construction...
296144    [Land Cleared, Land Cleared, Land Cleared, Lan...
296145    [Greenland, Greenland, Greenland, Greenland, C...
Length: 296146, dtype: object


## Area Filtering

### Computing the data

In [23]:
# Area of the building
train_area = np.asarray(train_df[['geometry']].area)

/var/folders/4c/2cw0jjvs6nb5v4xdxk3ty9s40000gn/T/ipykernel_19243/2481818807.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  train_area = np.asarray(train_df[['geometry']].area)


In [24]:
def calculate_convex_hull_area(geometry):
    try:
        convex_hull = geometry.convex_hull
        return convex_hull.area
    except:
        return 0

# Calculate the area of the convex hull for each geometry
train_convex_area = train_df['geometry'].apply(calculate_convex_hull_area)

# Convert to numpy array
train_convex_area = np.asarray(train_convex_area)

In [25]:
# Convex hull ratio
train_convex_ratio = train_area / train_convex_area
train_convex_ratio = train_convex_ratio.reshape(-1, 1)

### Analyze the area parameters

In [26]:
# Number of non convex areas
non_convex_areas_n = np.sum(train_convex_ratio < 0.99)
non_convex_areas_ratio = non_convex_areas_n / len(train_convex_ratio)
print('Number of non convex areas: ' + str(non_convex_areas_n))
print('Ratio of non convex areas of ' + str(non_convex_areas_ratio*100) + '%.')
mean_area = np.mean(train_area)

# Mean and Standard deviation of areas
std_area = np.std(train_area)
print('Mean area: ' + str(mean_area) + ' and std area: ' + str(std_area))

Number of non convex areas: 42369
Ratio of non convex areas of 14.306794621571791%.
Mean area: 5.792798723701533e-07 and std area: 6.544386914765251e-05


Coherent ! a road takes less place than a residential area.

## Stacking all the features

In [27]:
# Normalize area
train_area = (train_area - train_area.mean()) / train_area.std()
train_convex_ratio = (train_convex_ratio - train_convex_ratio.mean()) / train_convex_ratio.std()

# Stacking the features
train_area_features = np.hstack([train_area.reshape(-1, 1), train_convex_ratio])
print(train_area_features)

[[ 0.00363946  0.28623304]
 [-0.00213689  0.28623304]
 [ 0.00369309  0.28623304]
 ...
 [-0.00868493  0.28623304]
 [-0.00882395  0.28623304]
 [-0.00881304  0.28623304]]


## Concatenate all the tables and PCA

### Concatenation

In [28]:
# Concatenating the features
train_features = np.hstack([train_area_features, train_colors, train_intervals])
print(train_features.shape)

(296146, 36)


### Applying PCA

In [29]:
# Applying PCA
W = np.dot(train_features.T, train_features) # compute covariance matrix
eigval, eigvec = np.linalg.eig(W) # compute eigenvalues and eigenvectors of covariance matrix

# Sort eigenvalues and eigenvectors
idx = eigval.argsort()[::-1] # Sort eigenvalues
eigvec = eigvec[:,idx] # Sort eigenvectors according to eigenvalues
eigval = eigval[idx] # Sort eigenvalues

print(np.sum(eigval[:1]) / np.sum(eigval))

0.2718283777478106


### Select the best k to achieve 85% 

In [30]:
k = 1
while k < len(eigval) and (np.sum(eigval[:k])/np.sum(eigval)) < 0.85:   
    k += 1

print('k=' + str(k), '; percent=' + str(int(np.sum(eigval[:k]) / np.sum(eigval)*100)) + '%.')

k=12 ; percent=86%.


In [31]:
# Selecting the first k principal components
train_features = np.dot(train_features, eigvec[:,:k])
print(train_features.shape)
print(train_features)

(296146, 12)
[[-1.3431028   1.74403024 -0.66057054 ... -0.89569744 -2.68677199
   0.29551962]
 [-1.48372117  0.04724137 -0.71059037 ... -0.23448728 -2.93330396
   0.50741579]
 [-1.19749859  1.68838922 -0.85279169 ... -1.06727309 -2.43797418
   0.18657035]
 ...
 [ 5.15659743  0.51523952 -0.07278216 ... -0.13675119  0.14544733
  -0.10777262]
 [-1.3586172   0.82527781  0.01601422 ...  2.56692745  0.76149078
   0.16726591]
 [-2.50098859 -0.37138308 -0.28813019 ...  2.23041491  0.77871598
   0.16023501]]


## Save Train Data

In [32]:
# Save the train features
np.save('preprocessed_data.npy', train_features)

In [33]:
# Save the train labels
train_y = train_df['change_type'].apply(lambda x: change_type_map[x])
print(train_y)

np.save('preprocessed_labels.npy', train_df['change_type'].map(change_type_map).values)

0         1
1         1
2         1
3         1
4         0
         ..
296141    3
296142    2
296143    2
296144    2
296145    2
Name: change_type, Length: 296146, dtype: int64


## Save Test Data

In [34]:
# Adapt the test data to the PCA model
test_df = gpd.read_file('test.geojson', index_col=0)

DriverError: test.geojson: No such file or directory